In [2]:
__author__ = "me"
__date__ = "2015_10_13"

%pylab inline
import pandas as pd
import numpy as np
import geopandas as gp

import pylab as plt
import os

import requests
s = requests.get("https://raw.githubusercontent.com/Casyfill/CUSP_templates/master/Py/fbMatplotlibrc.json").json()
plt.rcParams.update(s)


Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [3]:
PARQA = os.getenv('PARQA')
path = PARQA + "data/Input/Parks_Data/PIP_InspectionMain.xlsx"

## Reading Data

In [4]:
df = pd.read_excel(path)

In [5]:
df = df[(pd.notnull(df.BeginInspection) ) &  (pd.notnull(df.EndInspection))]

## Parsing dates

In [7]:
df.head(2)

,Prop ID,Season,Round,Date,BeginInspection,EndInspection,Inspection Year,Overall Condition,Cleanliness,VisitorCount,Unacceptable Features,Inspection ID
0,M010-ZN08,Summer,2,2015-06-30,12:29:00,13:24:00,2015,A,A,60,SW,97371
1,M010-143,Summer,2,2015-06-30,13:40:00,13:57:00,2015,A,A,7,NaN,97370


In [9]:
df.Date = pd.to_datetime(df.Date, format='%Y-%m-%d')

In [21]:
def trySplit(x):
    if '-' in x:
        return x.split('-')[0]
    else:
        return x


In [22]:
df['Prop ID'] = df['Prop ID'].apply(trySplit)
df.head()

In [26]:
# g = df[['Date','Prop ID','Overall Condition']].groupby('Prop ID').agg(lambda x: x.tolist()).reset_index()
# g.head(2)

In [15]:
def inRange(d, inspection, marginDays=7):
    '''tells if date is withing margin before the inspection'''
    return inspection - datetime.timedelta(days = marginDays) <=  d <= inspection

In [ ]:
### ADD PROP ID to each call trhough calls
### aggregate Calls to Those ratings

In [28]:
inspections = df[['Prop ID','Date', 'Overall Condition', 'Cleanliness']]


## Read matched Calls

In [29]:
calls = pd.read_csv(PARQA + '/data/311/calls_matched_property.csv')

In [30]:
calls['Created Date'] = pd.to_datetime(calls['Created Date'])

AttributeError: Cannot access callable attribute 'to_dict' of 'DataFrameGroupBy' objects, try using the 'apply' method

In [ ]:
def aggCalls():
    ''''''

In [ ]:


parks = calls.groupby('ParkID')

d = {}
for name, g in parks:
    inspections = df[df['Prop ID']==name]
    
    calls = g[g['Created Date'].apply(lambda x: inRange(x,))]
    